In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [3]:
train = pd.read_csv('./data/final_train.csv.zip', compression='zip')
#test = pd.read_csv('./data/final_test.csv.zip', compression='zip')
target = np.load('./data/target.npy')
#submission_ids = np.load('./data/submission_ids.npy')

In [4]:
target = np.log1p(target)
#X_train, X_valid, Y_train, Y_valid = train_test_split(train, target, test_size=.15)

#train_xgb = xgb.DMatrix(data=train, label=target)
#valid_xgb = xgb.DMatrix(data=X_valid, label=Y_valid)
#train_lgb = lgb.Dataset(data=X_train, label=Y_train)
#valid_lgb = lgb.Dataset(data=X_valid, label=Y_valid)

In [14]:

# 	** see this https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst

# 1 - Controling model complexity :
# 	** max_depth, min_child_weight, and gamma
# 	** and those i will tune them for a large scale of values.

# 2 - control model's robusting to noise.
# 	** subsample and colsample_bytree

# 3 - Regularization terms.
# 	** alpha and lambda

# 4 - learning rate and number of rounds.
# 	** eta and num_rounds


params = {
    #'max_delta_step' : 0,
    #'scale_pos_weight' : 1, # calculated for each fold. #neg / #pos

    'max_depth' : 2,
    'min_child_weight' : 8,
    'gamma' : 0,

    'subsample' : .75,
    'colsample_bytree' : .75,

    'reg_alpha' : 1e-4,
    'reg_lambda' : 100,

    'eta' : 0.01,
    'n_estimators':1000,

    'objective' : "reg:linear",
    'eval_metric' : 'rmse',
    'n_jobs' : -1,
    'random_seed' : 10
    }

model = xgb.XGBRegressor(**params)
pg = {}


In [16]:
gs = GridSearchCV(model, param_grid=pg, scoring='neg_mean_squared_error', cv=5, verbose=1)
gs.fit(train, target)
#model = xgb.XGBRegressor(**params)


KeyboardInterrupt: 

In [ ]:
print(gs.best_params_)
gs.best_score_


In [ ]:
#cv_results = xgb.cv(params, train_xgb, num_boost_round=rounds, early_stopping_rounds=esrounds,
              #  seed=10, nfold=nfold, verbose_eval=20, metrics='rmse', show_stdv=False )
#model = xgb.XGBRegressor(params,n_estimators=10000000, learning_rate=0.005, gamma=0.0, subsample=0.75,
#                           colsample_bytree=1, n_jobs=-1, eval_metric='rmse')

In [ ]:
_ = model.fit(X_train, Y_train, eval_metric='rmse',  eval_set=[(X_valid,Y_valid)],
              early_stopping_rounds=50, verbose=20)

In [ ]:
model.

  -------------

In [ ]:
model_predictions_test = model.predict(test)
model_predictions_test

In [ ]:
model_predictions_test = np.expm1(model_predictions_test)
model_predictions_test

In [ ]:
dataset_submission = pd.DataFrame()
dataset_submission['ID'] = submission_ids
dataset_submission['target'] = model_predictions_test
dataset_submission.to_csv('./data/submission.csv', index=False)

---------------------------------------------------------

In [ ]:
#[99]	validation_0-rmse:1.40354
#[1413]	validation_0-rmse:1.38514